In [15]:
from IPython.display import Image
from qutip import *
import qutip.settings as qset
import latex
import pandas as pd
import time
import numpy as np
from scipy.interpolate import interp1d,CubicSpline
from scipy.interpolate import Rbf, InterpolatedUnivariateSpline
import multiprocessing
import concurrent.futures
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from palettable.colorbrewer.qualitative import Set1_5

plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.size": "15",
    "font.serif": ["Times New Roman"]})
import matplotlib
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14)
matplotlib.rc('legend', fontsize=13) 

In [16]:
# Parameters

kappa = 1.0 #cavity mode dissipation rate (this is why the parameters should be passed as arguments to the function func1)
Oc = 1.0*kappa #Rabi frequency 
delta = 0.5*kappa   # Delta_P
Nat = 1    
N = 12 #Fock space dimension
g = (1.0/np.sqrt(Nat))*kappa # List atom-field coupling strenght
EM = np.sqrt(0.01)*kappa #pump field strength
norm = 4*(EM/(kappa))**2 #normalization
#decay rates
G31 = 0.5*kappa  #atom
G32 = 0.5*kappa  #atom
#detunings
D1 = 0.0
D2 = 0.0
#D2 = kappa
t_list = np.linspace(0, 10, 100000)  #Time evolution

#Operators
a = tensor(destroy(N), qeye(3))
sm = tensor(qeye(N), sigmam())
sz = tensor(qeye(N), sigmaz())
S11 = tensor(qeye(N), basis(3,0) * basis(3,0).dag())
S22 = tensor(qeye(N), basis(3,1) * basis(3,1).dag())
S33 = tensor(qeye(N), basis(3,2) * basis(3,2).dag())
S13 = tensor(qeye(N), basis(3,0) * basis(3,2).dag())
S23 = tensor(qeye(N), basis(3,1) * basis(3,2).dag())
#alfa = tensor(qeye(N), qeye(2)) * (-1j * epsilon / kappa)

#Projectors
p1 = 1 * tensor(projection(N, 1, 1, offset = None), qeye(3))
p2 = 1 * tensor(projection(N, 2, 2, offset = None), qeye(3))
p3 = 1 * tensor(projection(N, 3, 3, offset = None), qeye(3))

In [17]:
#Coupled Hamiltonian

H = D1*S33 + (D1-D2)*S22 + delta*S11 - delta*a.dag()*a + (g*a*S13.dag() + Oc*S23.dag() + EM*a + g*a.dag()*S13 + Oc*S23 + EM*a.dag())

#Collapse operators

C = np.sqrt(kappa) * a #Cavity relaxiation

C31 = np.sqrt(G31) * S13 #Atomic polarization decay

C32 = np.sqrt(G32) * S23

#Collapse operators

c_ops = [C, C31, C32]   #talvez mudar para: [C] + C31 + C32

# Initial state (Ground-state)

aux_list = []
for m in range(Nat+1):
    aux_list.append(basis(3,1))
aux_list[0] = basis(N,0)
psi0 = tensor(aux_list)

In [19]:
#Solving the dynamics in monte carlo for 1 trajectorie

arg = [a.dag()*a, (a.dag()**2)*(a**2), S11, S22, S33, p1, p2, p3]

data = mcsolve(H, psi0, t_list, c_ops, arg, progress_bar=False, ntraj=1)

In [ ]:
#Calculate the expectation values for every rho_ss

n_p = data.expect[0]      #Mean number of photon inside the cavity
n2 = data.expect[1]       #to calculate g2
n11 = data.expect[2]      #Mean number of occupation in excited state |
n11 = data.expect[2]      #Mean number of occupation in excited state
n11 = data.expect[2]      #Mean number of occupation in excited state
g20 = n2/n_p**2           #Second-order correlation function 

P_1 = data.expect[3] 
P_2 = data.expect[4] 